In [1]:
import pickle as pickle
import pandas as pd
import numpy as onp
import bisect
import json
import itertools
from itertools import chain, combinations
import os


In [2]:
pars = {"lambda": [0.001],
            "sig":[0.5],
            "eta": [0.01],
            "beta": [0.1],
            "neta": [0.1],
            "nu": [10.0],
            "lu": [0.0],
            "lr": [0.1],
            "ot": ["freeZ"],
            "epchs": [500],
            "bs":[100],
            "reps":[1]}

In [3]:
repos = "/home/emiliano/Documents/ISP/postdoc/discoveringLatentConfounders/data/data_noisyproxy2/"

In [29]:
def getDataSetTab(repos, pars):
    files = os.listdir(repos)
    files = list(set(files).difference(set(["log.txt"])))
    parsList = [pars[k] for k in pars.keys()]
    combos = [list(it) for it in itertools.product(*parsList)]
    combos = [[combos[j][i] for j in range(len(combos))] for i in range(len(combos[0]))]
    combos = {k: v for k, v in zip(pars.keys(), combos)}
    
    aux = {"fileNames": files}
    aux = pd.DataFrame.from_dict(aux)
    datasetTab2 = {"fileNames": [f for f in files for i in range(len(combos["lambda"]))]}

    for par in pars.keys():
        datasetTab2[par] = [p for f in files for p in combos[par]]

    datasetTab2 = pd.DataFrame.from_dict(datasetTab2)
    datasetTab2 = datasetTab2.merge(aux, on="fileNames", sort=True)
    datasetTab2["job"] = onp.arange(1, datasetTab2.shape[0]+1)
    return datasetTab2

In [30]:
datasetTab = getDataSetTab(repos, pars)

In [31]:
datasetTab

,fileNames,lambda,sig,eta,beta,neta,nu,lu,lr,ot,epchs,bs,reps,job
0,rep100_gaussian1exponential1_independent_20_1_...,0.001,0.5,0.01,0.1,0.1,10.0,0.0,0.1,freeZ,500,100,1,1
1,rep100_gaussian1exponential1_independent_20_1_...,0.001,0.5,0.01,0.1,0.1,10.0,0.0,0.1,freeZ,500,100,1,2
2,rep100_gaussian1exponential1_independent_20_1_...,0.001,0.5,0.01,0.1,0.1,10.0,0.0,0.1,freeZ,500,100,1,3
3,rep100_gaussian1exponential1_independent_20_1_...,0.001,0.5,0.01,0.1,0.1,10.0,0.0,0.1,freeZ,500,100,1,4
4,rep100_gaussian1exponential1_independent_20_1_...,0.001,0.5,0.01,0.1,0.1,10.0,0.0,0.1,freeZ,500,100,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,rep9_gaussian1uniform1_independent_20_1_1000_1...,0.001,0.5,0.01,0.1,0.1,10.0,0.0,0.1,freeZ,500,100,1,1496
1496,rep9_gaussian1uniform1_independent_20_1_1000_1...,0.001,0.5,0.01,0.1,0.1,10.0,0.0,0.1,freeZ,500,100,1,1497
1497,rep9_gaussian1uniform1_independent_20_1_1000_1...,0.001,0.5,0.01,0.1,0.1,10.0,0.0,0.1,freeZ,500,100,1,1498
1498,rep9_gaussian1uniform1_independent_20_1_1000_5...,0.001,0.5,0.01,0.1,0.1,10.0,0.0,0.1,freeZ,500,100,1,1499


In [24]:
datasetTab.shape

(1500, 14)

In [8]:
onp.unique(datasetTab["fileNames"]).shape

(1500,)

In [9]:
nms = [datasetTab["fileNames"][i].split(".")[0] for i in range(datasetTab.shape[0])]

In [10]:
onp.unique(nms).shape

(1500,)

In [11]:
jobs = [732, 749]
jobs = [733, 750]

In [12]:
list(datasetTab.loc[datasetTab["job"].isin(jobs)]["fileNames"])

['rep97_gaussian1uniform1_independent_20_1_1000_50.csv',
 'rep11_gaussian1exponential1_independent_20_1_1000_500.csv']

In [16]:
datasetTab.loc[datasetTab["fileNames"].isin(["rep54_gaussian1exponential1_independent_20_1_1000_1000.csv"])]

,fileNames,lambda,sig,eta,beta,neta,nu,lu,lr,ot,epchs,bs,reps,job
706,rep54_gaussian1exponential1_independent_20_1_1...,0.001,0.5,0.01,0.1,0.1,10.0,0.0,0.1,freeZ,500,100,1,707


In [17]:
datasetTab.loc[datasetTab["job"].isin(jobs)]

,fileNames,lambda,sig,eta,beta,neta,nu,lu,lr,ot,epchs,bs,reps,job
732,rep97_gaussian1uniform1_independent_20_1_1000_...,0.001,0.5,0.01,0.1,0.1,10.0,0.0,0.1,freeZ,500,100,1,733
749,rep11_gaussian1exponential1_independent_20_1_1...,0.001,0.5,0.01,0.1,0.1,10.0,0.0,0.1,freeZ,500,100,1,750
